## Exploration of explicitly encoding Lexical Patterns into Neural Models

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%ls

In [ ]:
%ls ..

In [ ]:
import numpy as np
import h5py
import composition_learning
from gensim.models import KeyedVectors as embedding_model

Zunächst erstmal Zielnomen, Nomen, Adjektive aus tsv-File laden

In [ ]:
import csv
path_to_adj_noun_file = "../data/amod.train.filtered"
gnoun_noun_adj_list = [] #a list of lists, each sublist containing a goal-noun, noun, adjective triple
goal_noun_list = [] #only the goal nouns, needed for functions from composition_learning.py.
with open(path_to_adj_noun_file, "r") as tsv_file:
    tsv_reader = csv.reader(tsv_file, delimiter='\t')
    for row in tsv_reader:
        gnoun_noun_adj_list.append(row)
        goal_noun_list.append(row[0].upper())

Danach Word Embeddings aus Modell laden:

In [ ]:
path_to_word_embeddings = "../data/GoogleNews-vectors-negative300.bin"
vector_space = embedding_model.load_word2vec_format(path_to_word_embeddings, binary=True)

Daten in das richtige Format für das Keras-Modell bringen:

In [ ]:
training_data, training_labels, not_in_list = composition_learning.construct_data_and_labels(gnoun_noun_adj_list,
                                                                                             vector_space,
                                                                                             goal_noun_list,
                                                                                             verbosity = 0)

In [ ]:
f.close()
f = h5py.File('../data/lexical_training_data.hdf5', 'w')
t_data = f.create_dataset('training_data', data=training_data)
t_labels = f.create_dataset('training_labels', data=training_labels)

In [ ]:
f = h5py.File('../data/lexical_training_data.hdf5', 'r')
t_data = f["training_data"]
t_labels = f["training_labels"]
np.shape(t_data)
np.shape(t_labels)

In [ ]:
def training_data_generator(training_data, training_labels, batch_size):
    num_batches = int(np.floor(np.shape(training_data)[0] / batch_size))
    i=0
    while i < num_batches*batch_size:
        gen_output = (training_data[i:i+batch_size], training_labels[i:i+batch_size])
        #gen_output = ([training_data[i:i+batch_size,0,:],training_data[i:i+batch_size,1,:]], training_labels[i:i+batch_size])
        #print(i, np.shape(gen_output))
        i += batch_size
        if i >= (num_batches*batch_size): i = 0
        #print(i)
        yield gen_output
#TODO: nimmt insgesamt zu wenige samples pro epoche


In [ ]:
gen = training_data_generator(t_data, t_labels, batch_size=64)
print(np.shape(next(gen)[1]))

Das Modell trainieren. tensor_model bezeichnet das modell, indem adjektiv mit Tensor zusammen zu einer Matrix multipliziert wird, die dann mit Nomen-vektor multipliziert wird um wieder einen Vektor zu erhalten.
weighted_model steht für ein additives Modell mit Gewichtigunsmatrizen für je Adjetiv und Nomen.

In [ ]:
print(np.shape(gnoun_noun_adj_list))
print(len(not_in_list))
print(np.shape(training_data), np.shape(training_labels))

In [ ]:
tensor_model = composition_learning.train_model(training_data_generator(training_data, training_labels, batch_size=256), 
                                                samples_per_epoch = np.shape(training_data)[0],
                                                composition_mode = 'tensor_mult_identity', verbosity=0)

In [ ]:
weighted_model = composition_learning.train_model(training_data_generator(training_data, training_labels, batch_size=256), 
                                                samples_per_epoch = np.shape(training_data)[0],
                                                composition_mode = 'weighted_adj_and_noun_add_identity', verbosity=0)


In [ ]:
??composition_learning.train_model

In [ ]:
weighted_model.name = "weighted_model"
tensor_model.name = "tensor_model"

Modelle abspeichern für später(funktioniert irgendwie noch nicht, wahrscheinlich wegen eigener layerdefinition):

In [ ]:
%mkdir ../models/

In [ ]:
weighted_model.save_weights("../models/weighted_model_weights")
tensor_model.save_weights("../models/tensor_model_weights")

In [ ]:
%cd ../repeval2017/

Modelle wieder laden:

In [ ]:
tensor_model = composition_learning.create_model(composition_mode = 'tensor_mult_identity')
tensor_model.load_weights("../models/tensor_model_weights")
weighted_model = composition_learning.create_model(composition_mode = 'weighted_adj_and_noun_add_identity')
weighted_model.load_weights("../models/weighted_model_weights")
weighted_model.name = "weighted_model"
tensor_model.name = "tensor_model"

Um mit den Modellen Vektoren zu berechnen, tut man folgendes:

In [ ]:
goal_noun, noun, adj = gnoun_noun_adj_list[0]


goal_noun_tensor_vector = tensor_model.predict(np.asarray([[vectorspace[adj], vectorspace[noun]]]))[0, 0]
goal_noun_weighted_add_vector = weighted_model.predict(np.asarray([[vectorspace[adj], vectorspace[noun]]]))[0]

### Evaluierung

Lade Datensatz von hier: http://jair.org/media/3640/live-3640-6413-jair.txt

In [ ]:
%cd ../data/
!wget "http://jair.org/media/3640/live-3640-6413-jair.txt"
%ls
%cd ../repeval2017/

Evaluierungsdatensatz einlesen:

In [ ]:
import csv
path_to_eval_file = "../data/live-3640-6413-jair.txt"
data = []
with open(path_to_eval_file, "r") as tsv_file:
    tsv_reader = csv.reader(tsv_file, delimiter='|')
    i = 0    
    for row in tsv_reader:
        if row[0][0] != '#':
            data.append([])
            for elem in row:
                data[i].append(elem.strip())
            i+=1

training_data = data[0:680] #Split nach Angabe von Turney
test_data = data[680:]
print(len(training_data))
print(len(test_data))

In [ ]:
??complete_vector_space

In [ ]:
from scipy.spatial.distance import cosine
import sys

def compute_distances_to_candidates(model, vector_space, sample, complete_vector_space=False, topn=150):
    #print("Processing {}".format(sample))
    distances = []
    not_in_vector_space = []
    
    phrase = sample[0]
    adj,noun = phrase.split()
    # berechne phrasenvektor mit modell
    #print("Complete_vector_space = {}".format(complete_vector_space))
    try:      
        if model.name == 'weighted_model': phrase_vec = model.predict(np.asarray([[vector_space[adj], vector_space[noun]]]))[0]
        elif model.name == 'tensor_model': phrase_vec = model.predict(np.asarray([[vector_space[adj], vector_space[noun]]]))[0,0]
    except:
        not_in_vector_space.append([adj,noun])
        return distances
    
    if complete_vector_space:
        #print(np.shape(phrase_vec))
        distances = vector_space.most_similar(positive=[phrase_vec], topn=topn)
    elif not complete_vector_space:
        for i in range(1,len(sample)):
            # berechne vektoren für alle targets per look up
            try:
                target_vec = vector_space[sample[i]]
                distance = cosine(phrase_vec, target_vec)
                distances.append((sample[i], distance))
            except KeyError as e:
                distances.append((sample[i], np.infty))#wörter, die nicht im vektorraum enthalten sind, werden als unendlich weit weg behandelt
                not_in_vector_space.append(sample[i])
                #print("{} is not in vectorspace".format(sample[i]))
            
    #sortiere nach distanz
    sorted_distances = sorted(distances, key=lambda x:x[1], reverse=False)
    #print(sorted_distances)
    return sorted_distances
    
def eval_model(model, vector_space, test_data, mode='average_rank', k=1, topn=100):
    if mode == 'average_rank':
        ranks = []
        empty_list_counter = 0
        for sample in test_data:            
            distances = compute_distances_to_candidates(model, vector_space, sample)
            if distances:
                for i in range(0, len(distances)):
                    if distances[i][0] == sample[1]: #find rank of gold label
                        #print(distances)
                        ranks.append(i)
            else:
                empty_list_counter += 1
        print("Could not compute rank in {} of {} cases due to empty distance lists.\n(Probably due to words not appearing in the embedding space.)".format(empty_list_counter, len(test_data)))
        return np.mean(ranks)
    elif mode == 'average_rank_complete_vectorspace':
        ranks = []
        for sample in test_data:
            distances = compute_distances_to_candidates(model, vector_space, sample, complete_vector_space=True, topn=topn)
            if distances:
                for i in range(0, len(distances)):
                    if distances[i][0] == sample[1]: #find rank of gold label
                        #print(distances)
                        ranks.append(i)
                    else:
                        ranks.append(topn + 1)
            else:
                empty_list_counter += 1
        print("Could not compute rank in {} of {} cases due to empty distance lists.\n(Probably due to words not appearing in the embedding space.)".format(empty_list_counter, len(test_data)))
        return np.mean(ranks)
    elif mode == 'accuracy':
        correct_counter = 0.0
        empty_list_counter = 0
        for sample in test_data:
            distances = compute_distances_to_candidates(model, vector_space, sample)
            if distances: 
                if distances[0][0] == sample[1]: #gold label is rank 1
                    correct_counter += 1
                else:
                    correct_counter += 6/7
            else:
                empty_list_counter += 1
                
        print("Could not compute rank in {} of {} cases due to empty distance lists.(Probably due to words not appearing in the embedding space.)".format(empty_list_counter, len(test_data)))
        return (correct_counter / (len(test_data) - empty_list_counter))
    elif mode == 'precision':
        correct_counter = 0
        empty_list_counter = 0
        for sample in test_data:
            distances = compute_distances_to_candidates(model, vector_space, sample)
            if distances:
                k_nearest = [target for target, distance in distances]
                if sample[1] in k_nearest[:k]:
                    correct_counter += 1
            else:
                empty_list_counter += 1
        return (correct_counter / (len(test_data) - empty_list_counter))
                


In [25]:
print("--------------------------") 
print("{}: Average rank of the gold label: {}".format(weighted_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='average_rank')))
print("--------------------------")
print("{}: Accuracy of the model: {}".format(weighted_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='accuracy')))
print("--------------------------")
print("{}: Precision_at_1 of the model: {}".format(weighted_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='precision')))
print("--------------------------") 
print("{}: Precision_at_2 of the model: {}".format(weighted_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='precision', k=2)))
print("--------------------------") 
print("{}: Precision_at_3 of the model: {}".format(weighted_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='precision', k=3)))
print("--------------------------") 
print("{}: Average rank of the gold label: {}".format(tensor_model.name.upper(), eval_model(tensor_model, vector_space, test_data, mode='average_rank')))
print("--------------------------")
print("{}: Accuracy of the model: {}".format(tensor_model.name.upper(), eval_model(tensor_model, vector_space, test_data, mode='accuracy')))
print("--------------------------")
print("{}: Precision_at_1 of the model: {}".format(tensor_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='precision')))
print("--------------------------")
print("{}: Precision_at_2 of the model: {}".format(tensor_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='precision', k=2)))
print("--------------------------")
print("{}: Precision_at_3 of the model: {}".format(tensor_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='precision', k=3)))


--------------------------
Could not compute rank in 18 of 1500 cases due to empty distance lists.
(Probably due to words not appearing in the embedding space.)
WEIGHTED_MODEL: Average rank of the gold label: 2.8684210526315788
--------------------------
Could not compute rank in 18 of 1500 cases due to empty distance lists.(Probably due to words not appearing in the embedding space.)
WEIGHTED_MODEL: Accuracy of the model: 0.8575284364758198
--------------------------
WEIGHTED_MODEL: Precision_at_1 of the model: 0.002699055330634278
--------------------------
WEIGHTED_MODEL: Precision_at_2 of the model: 0.101889338731444
--------------------------
WEIGHTED_MODEL: Precision_at_3 of the model: 0.5364372469635628
--------------------------
Could not compute rank in 18 of 1500 cases due to empty distance lists.
(Probably due to words not appearing in the embedding space.)
TENSOR_MODEL: Average rank of the gold label: 2.9581646423751686
--------------------------
Could not compute rank in 1

In [ ]:
print("{}: Average rank of the gold label: {} (whole vectorspace!)".format(weighted_model.name.upper(), eval_model(weighted_model, vector_space, test_data, mode='average_rank_complete_vectorspace')))